In [29]:
import torch
import einops
from utils import *

In [20]:
B, Dq, Dv, Dm, h, H, W = (16, 256, 128, 256, 8, 32, 32)


def break_into_heads(M):
    D = M.shape[1]
    return M.reshape(B, h, D // h, H, W).flatten(0, 1)


Q = torch.randn(B, Dq, H, W)
K = torch.randn(B, Dq, H, W)
V = torch.randn(B, Dv, H, W)

Q = torch.nn.functional.softmax(Q, dim=-3)
K = torch.nn.functional.softmax(K.flatten(-2), dim=-1).reshape(B, Dq, H, W)

Q = break_into_heads(Q)
K = break_into_heads(K)  # (B * h, Dq // h, H, W)
V = break_into_heads(V)  # (B * h, Dv // h, H, W)

print("K shape:", K.shape, "V shape:", V.shape)

# K_bmm = K.flatten(-2, -1).transpose(1, 2)  # (B * h, HW, Dq // h)
# V_bmm = V.flatten(-2, -1).transpose(1, 2)  # (B * h, HW, Dv // h)
# KV_bmm = torch.bmm(K_bmm.permute(0, 2, 1), V_bmm)
# print(KV_bmm.shape)

# K_loop = K.flatten(-2, -1).transpose(1, 2)  # (B * h, HW, Dq // h)
# V_loop = V.flatten(-2, -1).transpose(1, 2)  # (B * h, HW, Dv // h)
# KV_loop = torch.zeros(KV_bmm.shape)

# # (B, D0, H, W), (B, D1, H, W) -> (B, D0, D1)
# # For each channel in D0, multiply it with each channel in D1
# for batch in range(K.shape[0]):
#     for k_channel in range(K.shape[1]):
#         for v_channel in range(V.shape[1]):
#             KV_loop[batch, k_channel, v_channel] = K_bmm[batch, :, k_channel].T @ V_bmm[batch, :, v_channel]

# K_vec = K  # .permute(0, 2, 3, 1) # (B * h, Dq // h, H, W)
# V_vec = V  # .permute(0, 2, 3, 1) # (B * h, Dv // h, H, W)
# KV_vec = torch.zeros(KV_bmm.shape)
# print(K_vec.shape, V_vec.shape)

# for batch in range(K_vec.shape[0]):
#     for k_channel in range(K_vec.shape[1]):
#         for v_channel in range(V_vec.shape[1]):
#             KV_vec[batch, k_channel, v_channel] = (K_bmm[batch, k_channel] * K_bmm[batch, v_channel]).sum()
# map_func = torch.vmap(torch.vmap(torch.dot))

# KV_vec = map_func(K_vec, V_vec)


KV_broadcast = (K.unsqueeze(2) * V.unsqueeze(1)).sum(dim=[-1, -2])

K_vmap = K
V_vmap = V


def mult_sum(A, B):
    AB = A * B
    return AB.sum()


batched_func = torch.vmap(torch.vmap(torch.vmap(mult_sum, in_dims=(None, 0)), in_dims=(0, None)), in_dims=(0, 0))
KV_vmap = batched_func(K_vmap, V_vmap)
print(KV_vmap.shape)

K shape: torch.Size([128, 32, 32, 32]) V shape: torch.Size([128, 16, 32, 32])
torch.Size([128, 32, 16])


In [45]:
def sum_pool_to_resolution(x, output_resolution=3):
    a = torch.nn.functional.interpolate(x, size=(output_resolution, output_resolution), mode="nearest")
    a = a * (output_resolution**2)
    return a


k = 3
KV_kernels = K.unsqueeze(2) * V.unsqueeze(1)  # (B * h, Dq // h, Dv // h, H, W)
KV_kernels = KV_kernels.flatten(0, 1)  # (B * h * Dq // h, Dv // h, H, W)
KV_kernels = sum_pool_to_resolution(KV_kernels, output_resolution=k)  # (B * h * Dq // h, Dv // h, k, k)
KV_kernels = KV_kernels.reshape(B * h, Dq // h, Dv // h, k, k)  # (B * h, Dq // h, Dv // h, k, k)